# Multi-Agent Collaboration

We will delve into the intricacies of agentic service and explore the strategies and best practices for cultivating effective multi-agent collaboration. We will examine how organizations can harness the collective intelligence of their agents, leverage emerging technologies to enhance coordination and communication, and navigate the challenges that often arise in complex, dynamic environments.

<img src="./images/multiagent.png" style="width:70%;display:block;margin: 0 auto;" />
<br />


In [ ]:
#!pip install crewai --force-reinstall
#!pip install --force-reinstall -v "setuptools<70"

In [ ]:
import os, sys
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv
from langchain.tools import DuckDuckGoSearchRun
module_paths = ["./", "../scripts"]
for module_path in module_paths:
    sys.path.append(os.path.abspath(module_path))
from bedrock import *
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

load_dotenv()
search_tool = DuckDuckGoSearchRun()
web_rag_tool = WebsiteSearchTool()

## Define agents

In [ ]:
class blogAgents():
    def planner(self, topic, model_id):
        return Agent(
            role="Content Planner",
            goal=f"Plan engaging and factually accurate content on {topic}",
            backstory="You're working on planning a blog article "
                      f"about the topic: {topic}."
                      "You collect information by searhing the web for the latest developements that directly relate to the {topic}."
                      "audience learn something "
                      "and make informed decisions. "
                      "Your work is the basis for "
                      "the Content Writer to write an article on this topic.",
            allow_delegation=False,
            tools=[search_tool, web_rag_tool],
            llm=get_llm(model_id),
            verbose=True
        )
        
    def writer(self, topic, model_id):
        return Agent(
            role="Content Writer",
            goal=f"Write insightful and factually accurate opinion piece about the topic: {topic}",
            backstory="You're working on a writing "
                      f"a new opinion piece about the topic: {topic}. "
                      "You base your writing on the work of "
                      "the Content Planner, who provides an outline "
                      "and relevant context about the topic. "
                      "You follow the main objectives and "
                      "direction of the outline, "
                      "as provide by the Content Planner. "
                      "You also provide objective and impartial insights "
                      "and back them up with information "
                      "provide by the Content Planner. "
                      "You acknowledge in your opinion piece "
                      "when your statements are opinions "
                      "as opposed to objective statements.",
            allow_delegation=False,
            llm=get_llm(model_id),
            verbose=True
        )

    def editor(self, model_id):
        return Agent(
            role="Editor",
            goal="Edit a given blog post to align with "
                 "the writing style of the organization. ",
            backstory="You are an editor who receives a blog post from the Content Writer. "
                      "Your goal is to review the blog post to ensure that it follows journalistic best practices,"
                      "provides balanced viewpoints when providing opinions or assertions, "
                      "and also avoids major controversial topics or opinions when possible.",
            allow_delegation=False,
            llm=get_llm(model_id),
            verbose=True
        )

## Define the tasks (plan, write and edit) for the agents we created above.

In [ ]:
class blogTasks():
    def plan(self, planner, topic):  
        return Task(
            description=(
                "1. Prioritize the latest trends, key players, "
                    f"and noteworthy news on {topic}.\n"
                "2. Identify the target audience, considering "
                    "their interests and pain points.\n"
                "3. Develop a detailed content outline including "
                    "an introduction, key points, and a call to action.\n"
                "4. Include SEO keywords and relevant data or sources."
            ),
            expected_output="A comprehensive content plan document "
                "with an outline, audience analysis, "
                "SEO keywords, and resources.",
            agent=planner,
        )
    def write(self, writer, topic):  
        return Task(
            description=(
                "1. Use the content plan to craft a compelling "
                    f"blog post on {topic}.\n"
                "2. Incorporate SEO keywords naturally.\n"
                "3. Sections/Subtitles are properly named "
                    "in an engaging manner.\n"
                "4. Ensure the post is structured with an "
                    "engaging introduction, insightful body, "
                    "and a summarizing conclusion.\n"
                "5. Proofread for grammatical errors and "
                    "alignment with the brand's voice.\n"
            ),
            expected_output="A well-written blog post "
                "in markdown format, ready for publication, "
                "each section should have 2 or 3 paragraphs.",
            agent=writer,
        )
        
    def edit(self, editor):
        return Task(
            description=("Proofread the given blog post for "
                         "grammatical errors and "
                         "alignment with the brand's voice."),
            expected_output="A well-written blog post in markdown format, "
                            "ready for publication, "
                            "each section should have 2 or 3 paragraphs.",
            agent=editor
        )

##  It’s time to assemble the crew. Combine the agents into our awesome crew.

In [ ]:
class blogCrew:
  def __init__(self, topic, model_id):
      self.topic = topic
      self.model_id = model_id

  def run(self):
    agents = blogAgents()
    tasks = blogTasks()

    planner_agent = agents.planner(self.topic, self.model_id)
    writer_agent = agents.writer(self.topic, self.model_id)
    editor_agent = agents.editor(self.model_id)

    plan_task = tasks.plan(planner_agent, self.topic)
    write_task = tasks.write(writer_agent, self.topic)
    edit_task = tasks.edit(editor_agent)


    crew = Crew(
        agents=[planner_agent, writer_agent, editor_agent],
        tasks=[plan_task, write_task, edit_task],
        verbose=True,
        memory=True,
        embedder={
            "provider": "huggingface",
            "config": {"model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"},
        },
        cache=True,
        process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is
    )

    result = crew.kickoff()
    return result

## Write a blog post based on the input topic

In [ ]:
topic = "Write a release note for Amazon Q"
model_id = "anthropic.claude-3-haiku-20240307-v1:0"
blog_crew = blogCrew(topic, model_id)
result = blog_crew.run()

## (Optional) Display the final output in markdown format 

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)